# Model Training

1. Retrieve all feature groups
2. Join features
3. Create a model per resort location (filter by location + sort)
4. Train XGBoost + validation
5. Check performance

In [1]:
import xgboost
import pandas as pd
import datetime
import hopsworks
import warnings
from locations import *
warnings.filterwarnings("ignore", module="IPython")


In [2]:
project = hopsworks.login(
    host="eu-west.cloud.hopsworks.ai",             # DNS of your Hopsworks instance
    project="ID2223_Project"
)

fs = project.get_feature_store()

2025-12-27 16:14:39,780 INFO: Initializing external client
2025-12-27 16:14:39,781 INFO: Base URL: https://eu-west.cloud.hopsworks.ai:443
2025-12-27 16:14:41,504 INFO: Python Engine initialized.

Logged in to project, explore it here https://eu-west.cloud.hopsworks.ai:443/p/2173


In [3]:
warning_fg = fs.get_feature_group(
    name='avalanche_warning',
    version=2
)

weather_fg = fs.get_feature_group(
    name="weather_sensor",
    version=1
)

terrain_fg = fs.get_feature_group(
    name="terrain_data",
    version=1
)

In [10]:
selected_features = (
    warning_fg
        .select(["location", "date", "warning_level"])
        .join(
            weather_fg.select([
                "temperature_2m_mean",
                "precipitation_sum",
                "rain_sum",
                "snowfall_sum",
                "wind_speed_10m_max",
                "wind_direction_10m_dominant"]),
            on=["location"]
        )
        .join(
            terrain_fg.select([
                "buffer_m",
                "mean_elevation",
                "std_elevation",
                "min_elevation",
                "max_elevation",
                "mean_slope",
                "std_slope",
                "steep_fraction_30deg",
                "steep_fraction_35deg"
            ]),
            on=["location"]
        )
)
selected_features.features


[Feature('location', None, None, False, False, False, None, None, None),
 Feature('date', None, None, False, False, False, None, None, None),
 Feature('warning_level', None, None, False, False, False, None, None, None),
 Feature('temperature_2m_mean', None, None, False, False, False, None, None, None),
 Feature('precipitation_sum', None, None, False, False, False, None, None, None),
 Feature('rain_sum', None, None, False, False, False, None, None, None),
 Feature('snowfall_sum', None, None, False, False, False, None, None, None),
 Feature('wind_speed_10m_max', None, None, False, False, False, None, None, None),
 Feature('wind_direction_10m_dominant', None, None, False, False, False, None, None, None),
 Feature('buffer_m', None, None, False, False, False, None, None, None),
 Feature('mean_elevation', None, None, False, False, False, None, None, None),
 Feature('std_elevation', None, None, False, False, False, None, None, None),
 Feature('min_elevation', None, None, False, False, False, 

In [11]:
feature_view = fs.create_feature_view(
    name="avalanche_warning_fv",
    version=1,
    description="Feature view combining avalanche warnings, weather forecasts, and static terrain features for Norwegian ski resorts.",
    query=selected_features,
    labels=["warning_level"]
)

Feature view created successfully, explore it at 
https://eu-west.cloud.hopsworks.ai:443/p/2173/fs/2122/fv/avalanche_warning_fv/version/1


In [ ]:
df = selected_features.read()
df.head()